### Archived Statements - part 4

Target:

 - us-east = 809,993,275
 - us-west = 809,993,275
 - archives = 3,239,979,516 + 2,159,986,344 + 2,159,986,344= 8,639,945,376

 
Process:

 - bulk import via stream consumer of import/statements_archive4
 - use externalScroller to bulk index at the same time

#### Run the imports

In [ ]:
!import.sh -o import -t 24 -l import/statements_archive4 -r archives -z 500 -b / -a -w 200000 -bulk -useScroller > logs/import-archives-step4.log

Started BulkIndex using ExternalScroller: aa27c114-50a1-452f-bb67-2f02df3381d9

In [2]:
# check status
! cid="aa27c114-50a1-452f-bb67-2f02df3381d9"; \
  INJECTOR="http://127.0.0.1:8080"; \
  curl -H 'Content-Type:application/json+nxrequest' -H 'X-NXRepository:archives' \
  -u $NXUSER:$NXPWD $INJECTOR/nuxeo/api/v1/bulk/$cid

{"entity-type":"bulkStatus","commandId":"aa27c114-50a1-452f-bb67-2f02df3381d9","state":"COMPLETED","processed":2160011807,"error":false,"errorCount":0,"total":2159986560,"action":"index","username":"nco-admin","submitted":"2020-09-22T15:22:05.068Z","scrollStart":"2020-09-22T15:22:05.086Z","scrollEnd":"2020-09-24T14:54:42.148Z","processingStart":null,"processingEnd":null,"completed":"2020-09-24T14:54:42.244Z","processingMillis":0}

In [3]:
from dateutil import parser
start = "2020-09-22T15:22:05.068Z"
end = "2020-09-24T14:54:42.148Z"
nbdocs = 2159986560
s = parser.parse(start)
e = parser.parse(end) 
throughput = nbdocs / (e-s).total_seconds()

print('%s docs/s' % format(throughput, ',.2f'))

12,619.91 docs/s


In [4]:
!mongo "mongodb+srv://nxbench-2826-archives.vzeqm.mongodb.net/nuxeo" \
  --username $MONGOUSER --password $MONGOPWD \
  ../ToolBox/archives-mongodb-stats.js > snapshots/"mongodb-archives-"`date +%Y%m%d-%H%M%S`.bson

In [5]:
!statfile="snapshots/es-7B-archive-indexing-"`date +%Y%m%d-%H%M%S`.txt; \
 curl "$ES_SERVER" > $statfile; \
 curl "$ES_SERVER/_cat/health?v" >> $statfile ; \
 curl "$ES_SERVER/_cat/nodes?v" >> $statfile; \
 curl "$ES_SERVER/_cat/indices?v" >> $statfile; \
 cat $statfile

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   509  100   509    0     0  20360      0 --:--:-- --:--:-- --:--:-- 20360
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   336  100   336    0     0   7304      0 --:--:-- --:--:-- --:--:--  7304
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1403  100  1403    0     0  19219      0 --:--:-- --:--:-- --:--:-- 19219
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   864  100   864    0     0  13090      0 --:--:-- --:--:-- --:--:-- 13090
{
  "name" : "G4PH9nZ",
  "cluster_name" : "18867088